In [10]:
import pickle
import torch
import pandas as pd
import random
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

In [11]:
checkpoint = "HuggingFaceTB/SmolLM2-360M-Instruct"

In [12]:
data_path = 'processed_data.pkl' 
with open(data_path, 'rb') as f:
    input_encodings, target_encodings = pickle.load(f)

In [13]:
device = "cpu" # for GPU usage or "cpu" for CPU usage

In [14]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint).to(device)

In [15]:
class TaskDataset(torch.utils.data.Dataset):
    def __init__(self, input_encodings, target_encodings):
        self.input_encodings = input_encodings
        self.target_encodings = target_encodings
    
    def __len__(self):
        return len(self.input_encodings.input_ids)
    
    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_encodings.input_ids[idx]),
            'attention_mask': torch.tensor(self.input_encodings.attention_mask[idx]),
            'labels': torch.tensor(self.target_encodings.input_ids[idx])  # Use target input_ids as labels
        }

In [16]:
random_indices = random.sample(range(len(input_encodings['input_ids'])), 100)

input_encodings_sample = {key: [value[i] for i in random_indices] for key, value in input_encodings.items()}
target_encodings_sample = {key: [value[i] for i in random_indices] for key, value in target_encodings.items()}

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    learning_rate=1e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=1000,
    save_total_limit=1,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,  # Set test dataset for evaluation
)


In [ ]:
trainer.train()

In [ ]:
model_save_path = 'task_subtask_model'
tokenizer.save_pretrained(model_save_path)
model.save_pretrained(model_save_path)